<a href="https://colab.research.google.com/github/C34021323/Portfolio_1-2/blob/main/%E4%BD%9C%E5%93%81%E9%9B%86_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **基於SVM跟CNN對人臉辨識的優劣比較**

SVM測試結果

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# 共1560張圖片；每張圖片高為100像素，寬為75像素，彩色
lfw_people = fetch_lfw_people(min_faces_per_person=50, resize=0.8, color=True)
print(lfw_people.images.shape)

x=lfw_people.data
print(x.shape)

y=lfw_people.target
names=lfw_people.target_names
print(names)

x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=42)

# 降維操作(減少特徵值):
# svd_solver=指定用哪種方法進行SVD計算；Randomized SVD=以隨機投影為基礎，快速抓取代表性的數據
# n_components=將資料降維到某個特徵值數量，並保有重要資訊
# whiten=白化處理，使各成分特徵變得獨立
pca = PCA(svd_solver='randomized', n_components=100, whiten=True)
pca.fit(x,y)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

# kernel=核函式，將二維上不可分割的資料映射到三維空間中，用適當的超平面予以分割
# rbf=徑向基核(Radial Basis Function)，是一種常用的非線性核函式
# C=正則化參數，數字越大，越可避免錯誤分類，但越有機會出現過擬合的情況
# gamma=控制SVM的決策邊界；值越大，模型越複雜，越易過擬合；值越小，模型越簡單，越易欠擬合
clf = SVC(kernel='rbf', C=100, gamma='auto', probability=True)
clf = clf.fit(x_train_pca, y_train)

score = clf.score(x_test_pca, y_test)
print('測試集準確率:', score)

for i in range(15):
  print('實際值:', names[y_test[i]], '預測值:', names[clf.predict([x_test_pca[i]])[0]])

In [ ]:
cm = confusion_matrix(y_test, clf.predict(x_test_pca))
print(cm)

# 混淆矩陣視覺化
plt.figure(figsize=(8, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

# 在格子標記數字+設定數字顏色
thresh = cm.max() / 2  # 閾值
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'), # 將矩陣數字渲染到格子中
                 ha='center', va='center',
                 color='white' if cm[i, j] > thresh else 'black') # 依條件給定不同顏色的數字

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(np.arange(cm.shape[1]))
plt.yticks(np.arange(cm.shape[0]))
plt.tight_layout()
plt.show()

CNN測試結果

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)

# 轉換函數可根據需求自定義
transform=transforms.Compose([
    transforms.ToTensor(),])

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people

lfw_people=fetch_lfw_people(min_faces_per_person=50, resize=0.8, color=True)

x=lfw_people.data # x 是模型的輸入特徵
y=lfw_people.target # y 是模型在訓練時用來學習和預測的輸出標籤

x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=42)

height, width, channels = 100, 75, 3
x_train=x_train.reshape(-1, channels, height, width)
x_test=x_test.reshape(-1, channels, height, width)

x_train=torch.tensor(x_train, dtype=torch.float32)
x_test=torch.tensor(x_test, dtype=torch.float32)
y_train=torch.tensor(y_train, dtype=torch.long)
y_test=torch.tensor(y_test, dtype=torch.long)

train_dataset=torch.utils.data.TensorDataset(x_train, y_train)
test_dataset=torch.utils.data.TensorDataset(x_test, y_test)

train_loader=DataLoader(train_dataset, batch_size=96, shuffle=True)
test_loader=DataLoader(test_dataset, batch_size=24, shuffle=False)

print(x_train.shape)
print(x_test.shape)

In [ ]:
# 定義CNN模型
class ConvNet(nn.Module):

  def __init__(self):
        super(ConvNet, self).__init__()
        self.cn1 = nn.Conv2d(3, 16, 3, 1)
        self.cn2 = nn.Conv2d(16, 32, 3, 1)
        self.dp1 = nn.Dropout(0.1)
        self.dp2 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(48*35*32, 64)  # fc1的輸入尺寸
        self.fc2 = nn.Linear(64, 12)  # 最後的輸出層

  def forward(self, x):
    x=self.cn1(x)
    x=F.relu(x)
    x=self.cn2(x)
    x=F.relu(x)

# 每 2x2 區域會取最大的值來代表該區域
    x=F.max_pool2d(x, 2)
# 展平(flatten):將經過卷積層和池化層後的特徵圖轉換為一維向量，從而使其能夠作為FC的輸入，進而分類或迴歸
    x=torch.flatten(x, 1)

    x=self.fc1(x)
    x=self.dp1(x)
    x=F.relu(x)

    x=self.fc2(x)
    x=self.dp2(x)
    output=F.log_softmax(x, dim=1)
    return output

# 建立CNN模型物件
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=ConvNet().to(device)

# 定義損失函數及最佳化方法
myloss=nn.NLLLoss() # 配合log_softmax()的損失函數
optim=torch.optim.Adadelta(model.parameters(), lr=0.5)

batch_size=96

# 定義train()副程式，用來訓練模型
def train(model, device, train_loader, myloss, optim, epoch): # train_loader=自動將數據分成數個batch，批次載入

  model.train()
  total_samples = len(train_loader.dataset)  # 總樣本數量

# 訓練模型的過程經常是由一個大迴圈(epoch)+一個小迴圈(batch training)組成
# 對dataloader中每個batch資料進行迴圈操作；b_i是batch的索引；(X, y)=每個batch中的輸入/ 標籤
  for b_i, (X, y) in enumerate(train_loader):
    X, y=X.to(device), y.to(device)
    pred_prob=model(X) # 實際調用forward方法，執行前向傳播
    loss=myloss(pred_prob, y)

    optim.zero_grad()
    loss.backward()
    optim.step()

    if b_i%1==0:
      num1=b_i*len(X)
      num2=len(train_loader.dataset)
      num3=100*b_i/len(train_loader)
      print(f'Train Epoch: {epoch} [{num1}/{num2} ({num3:.0f}%)]\tLoss: {loss.item():.6f}')

epochs=3
for epoch in range(epochs):
  train(model, device, train_loader, myloss, optim, epoch)

torch.save(model.state_dict(), 'mnist_cnn.pt') # 儲存模型參數，以字典形式儲存，日後不必重新訓練

In [ ]:
model2=ConvNet()
model2.load_state_dict(torch.load('mnist_cnn.pt')) # 引用儲存的模型，繼續訓練
model2=model2.to(device)

# 定義test()副程式，用來測試模型
def test(model, device, test_loader, myloss): # test_loader=自動將數據分成數個batch，批次載入

  model.eval()
  loss=0
  success=0 # 從0開始，逐步累加正確預測次數

# 預測階段時，模型的參數（權重和偏差）應該保持固定，不應該被更新
  with torch.no_grad():
    for X, y in test_loader:
      X, y=X.to(device), y.to(device)
      pred_prob=model(X)
      loss+=myloss(pred_prob, y).item()

# 計算模型預測正確次數
      pred=pred_prob.argmax(dim=1, keepdim=True) # argmax()可找出最大tensor的index
      success+=pred.eq(y.view_as(pred)).sum().item() # eq()可比較tensor是否相等

      num1=loss/len(test_loader)
      num2=len(test_loader.dataset)
      num3=100*success/len(test_loader.dataset)
      print('Overall Loss:{:.4f}, overall Accuracy: {}/{}({:.2f}%)'.format(num1, success, num2, num3))

test(model2, device, test_loader, myloss)